In [155]:
###Pandas Libraries
import pandas as pd
import datetime
import time

#Dash Libraries
#import dash  # USE THIS IF RUNNING ON SERVER
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash # USE THIS IF RUNNING ON JUPYTER
import numpy as np

In [241]:
#*********                     FUNCTION RUNS ALL OF THE DATA MANIPULATION CODE
## TIMER FUNCTION
start = time.time()

mt_beds = 13 #Current Number of Beds Available in Maternity
ds_beds = 4  #Current Number of Beds Available in Delivery Suite

## Bring in Transfers data, update dates and remove uneeded columns
df = pd.read_csv('transfers.csv')
df = df.rename(columns = {'Start_Bed_Dttm':'Start', 'End_Bed_Dttm':'End'})
df = df.drop(columns=['Transfers', 'Bed_Code', 'Full Flow Code', 'Current Flow Code', '1st Level','PASID'])
date_cols = ['Start', 'End']
df[date_cols] = df[date_cols].apply(pd.to_datetime, format='%d/%m/%y %H:%M', errors='raise')
#df[date_cols] = df[date_cols].apply(lambda x: x.dt.strftime('%d/%m/%Y %H:%M:%S'))

#MATERNITY
## Make new dataframe patients entering Maternity Ward
mt_in_df= pd.DataFrame()
mt_in_df['Event'] = df.loc[df['Ward_Code'] == 'MT', 'Start']
mt_in_df['Patient_Change'] = 1
## make new dataframe with patients leaving room
mt_out_df= pd.DataFrame()
mt_out_df['Event'] = df.loc[df['Ward_Code'] == 'MT', 'End']
mt_out_df['Patient_Change'] = -1
## Concatenate the two datasets and sort by date to determin current patinet occupancies
mt_count_df = pd.concat([mt_in_df, mt_out_df])
mt_count_df.sort_values('Event', ascending = True, inplace=True)
mt_count_df['MT_Patients'] = mt_count_df['Patient_Change'].cumsum()
mt_count_df['Bed_Available'] = mt_count_df['MT_Patients'] <= mt_beds
mt_count_df.dropna(inplace = True)

# DELIVERY SUITE(DS)
## Make new dataframe patients entering room
ds_in_df = pd.DataFrame()
ds_in_df['Event'] = df.loc[df['Ward_Code'] == 'DS', 'Start']
ds_in_df['Patient_Change'] = 1
## make new dataframe with patients leaving room
ds_out_df= pd.DataFrame()
ds_out_df['Event'] = df.loc[df['Ward_Code'] == 'DS', 'End']
ds_out_df['Patient_Change'] = -1
## Concatenate the two datasets and sort by date to determin current patinet occupancies
ds_count_df = pd.concat([ds_in_df, ds_out_df])
ds_count_df.sort_values('Event', ascending = True, inplace=True)
ds_count_df['DS_Patients'] = ds_count_df['Patient_Change'].cumsum()
ds_count_df['Bed_Available'] = ds_count_df['DS_Patients'] <= mt_beds
ds_count_df.dropna(inplace = True)

# CLEAN DATA TO PRODUCE IDEAL PATIENT TRANSFERS
ideal_df = pd.DataFrame(df)
#CA, CL & HE Codes are converted to MT
ideal_df.loc[(ideal_df['Ward_Code'] == 'CA') | (ideal_df['Ward_Code'] == 'CL') | (ideal_df['Ward_Code'] == 'HE') , 'Ward_Code'] = 'MT' #Ward Code CA = MT
## Drop DPS from the new dataset
index_names = ideal_df.loc[(ideal_df['Ward_Code'] == 'DPS') | ((ideal_df['Ward_Code'] == 'ON'))].index
ideal_df.drop(index_names, inplace = True)
## Groupby the event code and with ward to summarise and finds any duplicates
events = ideal_df.groupby(['Link', 'Ward_Code'])
#events.get_group(('ADE-272865','MT'))
#get the min start date and max end date from the grouped results
ideal_df['min_start'] = events['Start'].transform('min')
ideal_df['max_end'] = events['End'].transform('max')
#Drop the Original start & end dates & rename the new calculated start & end
ideal_df.drop(columns=['Start', 'End'], inplace = True)
ideal_df.rename(columns= {'min_start':'Start', 'max_end': 'End'}, inplace = True)
ideal_df.sort_values(['Link', 'Ward_Code'], ascending = True, inplace = True)
ideal_df.drop_duplicates(keep='first', inplace = True)
#checks that there are no longer any duplicates
#ideal_df.loc[ideal_df.duplicated(keep=False), :]

# CALCULATE LENGTH OF STAY
ideal_df['LOS_hrs'] = (ideal_df['End'] - ideal_df['Start']).dt.seconds / (60*60)

# EXTRACT EVENTS DATA AND PUT IN THE IDEALISED DATA TABLE
events_df = pd.read_csv('exception_dates.csv')
#events_df.drop(columns=['Unnamed: 2'], inplace = True)
events_df.rename(columns = {'DATE':'Date', 'EVENT':'Event'}, inplace=True)
events_df['Date'] = events_df['Date'].apply(pd.to_datetime, format='%d/%m/%y', errors='raise')
events_df['Date'] = events_df['Date'].dt.date
events_df.sort_values(['Event', 'Date'])
## Combine all labels with the same date into the same field
grouped_events_df = events_df.groupby(['Date'])['Event'].apply(','.join).reset_index()
grouped_events_df.sort_values('Event', ascending=True)
## Delete erroneous Event Data
grouped_events_df = grouped_events_df.drop(grouped_events_df[grouped_events_df.Event == 'SCHOOL,SCHOOL'].index)
grouped_events_df = grouped_events_df.drop(grouped_events_df[grouped_events_df.Event == 'PUBLIC,SCHOOL,SCHOOL'].index)
## Merge data from the events table back into the Idealised data table
ideal_df['Date'] = ideal_df['Start'].dt.date
ideal_merged_df = pd.merge(grouped_events_df, ideal_df, how='right', on='Date')

end = time.time()
print(str(round(end - start,1)) + ' seconds to run code.')

0.4 seconds to run code.


In [248]:
#import dash  # USE THIS IF RUNNING ON SERVER
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash # USE THIS IF RUNNING ON JUPYTER
import numpy as np

#app = dash.Dash(__name__) # USE THIS IIF RUNNING ON SERVER
app = JupyterDash(__name__) # USE THIS IF RUNNING ON JUPYTER

app.layout = html.Div([
    
    html.Div([
        html.Pre(children= " Length of Stay Histogram Plot",
                style={"text-align": "center", "font-size": "100%", "color":"black"})
    ]),
    
    dcc.Dropdown(
        id='my_dropdown',
        options=[
            {'label': 'Maternity', 'value': 'MT'},
            {'label': 'Delivery Suite', 'value': 'DS'},
            {'label': 'ICU', 'value': 'ICU'},
                ],
                 optionHeight=35,
                 value='MT',
                 disabled=False,
                 multi=False,
                 searchable=True,
                 search_value='',
                 placeholder='Please select... ',
                 clearable=True
                 #style={'width':"100%"},
                 #className='select_box',
                 #persistence=True,
                 #persistence_type='memory' #'session','local' 
                ),
    
    dcc.Graph(id='the_graph')
    
])

@app.callback(
    Output(component_id ='the_graph', component_property='figure'), 
    [Input(component_id ='my_dropdown', component_property='value')])
def update_graph(my_dropdown):
    plot_df = ideal_df
    plot_df = plot_df.loc[plot_df['Ward_Code'] == my_dropdown, :]
    fig = px.histogram(plot_df, x="LOS_hrs", nbins=50)
    return fig

#app.run_server(debug=True)  # USE THIS IF RUNNING ON SERVER
app.run_server(mode='inline') # USE THIS IF RUNNING ON JUPYTER

In [242]:
plot_df = ideal_df
ideal_df
#plot_df['LOS'] = plot_df.LOS.dt.seconds / (60*60)
#plot_df

,Link,Ward_Code,Start,End,LOS_hrs,Date
1,ADE-271165,MT,2013-06-30 22:21:00,2013-07-04 10:40:00,12.316667,2013-06-30
4,ADE-271206,DS,2013-06-27 12:40:00,2013-06-28 10:20:00,21.666667,2013-06-27
5,ADE-271206,MT,2013-06-28 10:20:00,2013-07-03 09:59:00,23.650000,2013-06-28
12,ADE-271551,MT,2013-06-27 23:54:00,2013-07-01 09:25:00,9.516667,2013-06-27
14,ADE-271568,DS,2013-07-11 07:16:00,2013-07-12 13:59:00,6.716667,2013-07-11
...,...,...,...,...,...,...
2027,ADE-510071,DS,2020-07-18 09:51:00,2020-07-19 11:45:00,1.900000,2020-07-18
2029,ADE-510071,MT,2020-07-19 11:45:00,2020-07-23 11:50:00,0.083333,2020-07-19
1522,ADE-510160,DS,2020-07-20 22:00:00,2020-07-21 13:15:00,15.250000,2020-07-20
3060,ADE-510160,MT,2020-07-21 15:20:00,2020-07-25 10:45:00,19.416667,2020-07-21


# Maternity Service Capacity - Model, Simulation & Analysis
## Objective 1
**Model current Maternity Service activity using process mapping and queuing network analysis**
1. Data Required
2. Profile - Patient Process Flow - Historical
3. Profile - Patient Process Flow - Idealised
4. Profile - Length of Stay
5. Profile - Arrival Rate
7. Variability Analysis - Events of Significance
8. Variability Analysis - Patient Process Flow
9. Variability Analysis - Length of Stay
10. Variability Analysis - Arrival Rate


## Objective 2
**Determine current capacity requirements using simulation and mathematical modeling**
1. Current Capacity Requirements - Historical Data
2. Current Capacity Requirements - Idealised Historical Data
3. Current Capacity Requirements - Simulated Model
4. Results Analysis - Accuracy with Idealised Historical Data

## Objective 3
**Determine future capacity requirements at 5 and 10 years based on population demographics projections data.**
1. Capacity Requirements - 5 year forecast
2. Capacity Requirements - 10 year forecast

***

## Objective 1
### 1.1. Data Required
#### 1.1.1 Historical Data
7 years of patient transfers data provided by hospital.
Data will provide the foundation of the mathematical model & data analysis

In [157]:
df

,Link,Ward_Code,Start,End
0,ADE-363169,MT,2016-02-22 05:30:00,2016-02-27 11:45:00
1,ADE-271165,MT,2013-06-30 22:21:00,2013-07-04 10:40:00
2,ADE-461781,ICU,2019-05-09 18:17:00,2019-05-10 13:24:00
3,ADE-501083,DS,2020-06-20 20:00:00,2020-06-22 03:38:00
4,ADE-271206,DS,2013-06-27 12:40:00,2013-06-28 10:20:00
...,...,...,...,...
8921,ADE-348899,MT,2016-02-15 14:27:00,2016-02-19 10:15:00
8922,ADE-491342,DS,2019-12-15 20:00:00,2019-12-16 13:24:00
8923,ADE-348904,MT,2016-04-05 22:51:00,2016-04-09 09:45:00
8924,ADE-425431,DS,2018-01-29 20:30:00,2018-01-31 11:27:00


#### 1.1.2  Critical Capacity Data
Key data to hospital Capacity Analysis

In [158]:
mt_beds = 13 #Current Number of Beds Available in Maternity
ds_beds = 4  #Current Number of Beds Available in Delivery Suite

### 1.4 Profile Length of Stay

#### 1.4.2 Profile of Length of Stay by Ward

In [159]:
ideal_df.groupby('Ward_Code').describe()

LOS                                                      \
          count                      mean                       std   
Ward_Code                                                             
DS         2339 0 days 22:19:06.618212911 0 days 12:09:48.296435046   
ICU          81 1 days 06:26:45.185185185 0 days 20:24:22.634704846   
MT         5783 3 days 23:55:35.929448383 1 days 15:17:05.855380638   

                                                                           \
                      min             25%             50%             75%   
Ward_Code                                                                   
DS        0 days 00:01:00 0 days 14:33:00 0 days 22:15:00 1 days 04:30:00   
ICU       0 days 02:34:00 0 days 17:53:00 0 days 23:59:00 1 days 15:17:00   
MT        0 days 01:30:00 3 days 04:25:00 3 days 20:38:00 4 days 18:36:00   

                            
                       max  
Ward_Code                   
DS         5 days 22:19:00  
ICU        6 days 15:52:00  
MT        34 days 19:50:00

#### 1.4.2 Profile of Length of Stay Maternity

In [160]:
ideal_df['LOS'][ideal_df['Ward_Code'] == 'MT'].value_counts(bins=10, normalize=True)*100

(3 days 12:56:00, 7 days 00:22:00]              65.819502
(0 days 00:39:56.399999999, 3 days 12:56:00]    32.365145
(7 days 00:22:00, 10 days 11:48:00]              1.210235
(10 days 11:48:00, 13 days 23:14:00]             0.311203
(13 days 23:14:00, 17 days 10:40:00]             0.103734
(17 days 10:40:00, 20 days 22:06:00]             0.069156
(31 days 08:24:00, 34 days 19:50:00]             0.034578
(27 days 20:58:00, 31 days 08:24:00]             0.034578
(24 days 09:32:00, 27 days 20:58:00]             0.017289
(20 days 22:06:00, 24 days 09:32:00]             0.017289
Name: LOS, dtype: float64

#### 1.4.3 Profile of Length of Stay Maternity

In [161]:
ideal_df['LOS'][ideal_df['Ward_Code'] == 'DS'].value_counts(bins=10, normalize=True)*100

(0 days 14:14:48, 1 days 04:28:36]                50.705430
(-1 days +23:52:27.719999999, 0 days 14:14:48]    23.899102
(1 days 04:28:36, 1 days 18:42:24]                21.633177
(1 days 18:42:24, 2 days 08:56:12]                 2.607952
(2 days 08:56:12, 2 days 23:10:00]                 0.726806
(2 days 23:10:00, 3 days 13:23:48]                 0.213767
(3 days 13:23:48, 4 days 03:37:36]                 0.128260
(5 days 08:05:12, 5 days 22:19:00]                 0.042753
(4 days 03:37:36, 4 days 17:51:24]                 0.042753
(4 days 17:51:24, 5 days 08:05:12]                 0.000000
Name: LOS, dtype: float64

### 1.6 Variability Analysis - Events of Significance
#### 1.6.1 Selected Events & Associated Dates

In [162]:
grouped_events_df.set_index('Date', inplace = True)
grouped_events_df.sample(frac=0.1)

,Event
Date,
2019-01-18,SCHOOL
2019-12-22,"SCHOOL,BEFOREXMAS"
2024-07-17,SCHOOL
2020-12-18,SCHOOL
2022-01-01,"PUBLIC,SCHOOL"
...,...
2020-04-18,SCHOOL
2021-04-29,SCHOOL
2020-07-11,SCHOOL


#### 1.6.2 Number of days in Dataset that these dates occur

In [163]:
grouped_events_df['Count'] = 1
grouped_events_df.groupby(['Event'], as_index=False)['Count'].count().sort_values('Count', ascending = False)

,Event,Count
13,SCHOOL,912
16,"SCHOOL,BEFOREXMAS",78
14,"SCHOOL,AFTERXMAS",63
1,B4EASTER,56
8,"PUBLIC,SCHOOL",47
2,BEFOREXMAS,46
3,"BEFOREXMAS,SCHOOL",44
5,PUBLIC,34
0,"AFTERXMAS,SCHOOL",30
7,"PUBLIC,EASTER",23


#### 1.6.3 Events of Significance brought back into idealised data

In [164]:
ideal_merged_df['Event'].fillna('NORMAL', inplace=True)
ideal_merged_df
#ideal_merged_df['Event'].notnull().sum()

,Date,Event,Link,Ward_Code,Start,End,LOS
0,2013-06-30,NORMAL,ADE-271165,MT,2013-06-30 22:21:00,2013-07-04 10:40:00,3 days 12:19:00
1,2013-06-30,NORMAL,ADE-271746,MT,2013-06-30 12:00:00,2013-07-03 09:58:00,2 days 21:58:00
2,2013-06-30,NORMAL,ADE-282352,MT,2013-06-30 13:00:00,2013-07-05 09:25:00,4 days 20:25:00
3,2013-06-27,NORMAL,ADE-271206,DS,2013-06-27 12:40:00,2013-06-28 10:20:00,0 days 21:40:00
4,2013-06-27,NORMAL,ADE-271551,MT,2013-06-27 23:54:00,2013-07-01 09:25:00,3 days 09:31:00
...,...,...,...,...,...,...,...
8199,2020-06-07,NORMAL,ADE-506055,MT,2020-06-07 17:48:00,2020-06-10 13:47:00,2 days 19:59:00
8200,2020-06-07,NORMAL,ADE-506091,DS,2020-06-07 23:45:00,2020-06-08 10:30:00,0 days 10:45:00
8201,2020-06-13,NORMAL,ADE-506671,DS,2020-06-13 20:00:00,2020-06-14 12:30:00,0 days 16:30:00
8202,2020-07-12,SCHOOL,ADE-508881,DS,2020-07-12 21:30:00,2020-07-13 17:22:00,0 days 19:52:00


## Objective 2
### 2.1 Current Capacity Requirements - Historical Data
#### 2.1.1 Maternity Transformed Data
Historical data of number of patients in Maternity at the same time

In [165]:
mt_count_df

,Event,Patient_Change,MT_Patients,Bed_Available
813,2013-06-27 16:00:00,1,1,True
12,2013-06-27 23:54:00,1,2,True
827,2013-06-28 06:30:00,1,3,True
5,2013-06-28 10:20:00,1,4,True
85,2013-06-28 12:27:00,1,5,True
...,...,...,...,...
4741,2020-07-29 15:58:00,-1,3,True
2274,2020-07-29 23:45:00,1,4,True
7819,2020-07-30 09:52:00,-1,3,True
6456,2020-07-31 10:04:00,-1,2,True


#### 2.1.2 Maternity Occupancy Profile

In [166]:
mt_count_df['MT_Patients'].describe()

count    11957.000000
mean         8.669482
std          2.515389
min          0.000000
25%          7.000000
50%          9.000000
75%         11.000000
max         15.000000
Name: MT_Patients, dtype: float64

#### 2.1.3 Delivery Suite Transformed Data

In [167]:
ds_count_df

,Event,Patient_Change,DS_Patients,Bed_Available
4,2013-06-27 12:40:00,1,1,True
4,2013-06-28 10:20:00,-1,0,True
44,2013-06-29 03:00:00,1,1,True
44,2013-06-30 12:00:00,-1,0,True
899,2013-07-03 21:04:00,1,1,True
...,...,...,...,...
4757,2020-07-24 19:30:00,-1,0,True
418,2020-07-24 19:47:00,1,1,True
7818,2020-07-25 16:21:00,1,2,True
418,2020-07-25 20:08:00,-1,1,True


#### 2.1.4 Delivery Suite Occupancy Profile

In [168]:
ds_count_df['DS_Patients'].describe()

count    4720.000000
mean        1.388136
std         1.022002
min         0.000000
25%         1.000000
50%         1.000000
75%         2.000000
max         5.000000
Name: DS_Patients, dtype: float64

### 2.2 Current Capacity Requirements - Idealised Historical Data
**Create new ideal patient flow records with existing data and clean the dataset**

In an ideal scenario, rather than sending patients to St Clares, St Helens and St Catherines they are sent to Maternity.  This process adjusts the actual data to what the ideal data would be.

As DPS is not relevant in this study we would also like to remove these entries from the dataset.

Once the data has been converted to ideal there are entries where the patient is transferred to the same room they left.  To clean the data we need to find the start date of when they first arrvied in the room and the end date when they left.

Once found we need to remove the duplicate entries and only show the first start date and last end date.

In [169]:
ideal_df

,Link,Ward_Code,Start,End,LOS,Date
1,ADE-271165,MT,2013-06-30 22:21:00,2013-07-04 10:40:00,3 days 12:19:00,2013-06-30
4,ADE-271206,DS,2013-06-27 12:40:00,2013-06-28 10:20:00,0 days 21:40:00,2013-06-27
5,ADE-271206,MT,2013-06-28 10:20:00,2013-07-03 09:59:00,4 days 23:39:00,2013-06-28
12,ADE-271551,MT,2013-06-27 23:54:00,2013-07-01 09:25:00,3 days 09:31:00,2013-06-27
14,ADE-271568,DS,2013-07-11 07:16:00,2013-07-12 13:59:00,1 days 06:43:00,2013-07-11
...,...,...,...,...,...,...
2027,ADE-510071,DS,2020-07-18 09:51:00,2020-07-19 11:45:00,1 days 01:54:00,2020-07-18
2029,ADE-510071,MT,2020-07-19 11:45:00,2020-07-23 11:50:00,4 days 00:05:00,2020-07-19
1522,ADE-510160,DS,2020-07-20 22:00:00,2020-07-21 13:15:00,0 days 15:15:00,2020-07-20
3060,ADE-510160,MT,2020-07-21 15:20:00,2020-07-25 10:45:00,3 days 19:25:00,2020-07-21
